<a href="https://colab.research.google.com/github/BethGomez44/proyectoDataScience/blob/main/ProyectoDataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Módulo 2. Análisis de Datos con Python
#### **Tema:** Recomendación de canciones usando python
#### **Objetivo:** Crear una lista de canciones que le pueden gustar al usuario a partir de su canción favorita de una base de datos.
##### **Integrantes:**
-
- Heidi Lizbeth Gómez de la Torre


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importamos todo lo que vamos a usar (libraries, datasets)
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity


df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/songs_normalize.csv")
df=pd.DataFrame(df)

In [ ]:
#convertimos los nombres de las canciones de objects a strings
#df= df['song'] = df['song'].astype("string")
#limpiamos nuestro dataset deshaciendonos de las columnas que no nos sirven y eliminando las canciones con populardad menor a 35, ya que es probable que estas no le gusten a nuestro usuario
dflimpio=df.drop(["danceability", "key", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence"], axis=1)
dflimpio = dflimpio[dflimpio["popularity"] >= 35]

In [ ]:
#pedimos la canción favorita del usuario
cancion_fav = str(input("Ingresa tu canción favorita, o ingrese 'Ver menu' para ver una muestra de canciones disponibles \n ¡Recuerda respetar las mayúsculas y los espacios de tu canción!\n "))

#checamos si esta cancion se encuentra en nuestro dataset
cancion = dflimpio['song'].str.contains(cancion_fav, case=False, na=False)

def pedir_cancion():
  if cancion.any():
     print("Excelente, en un momento te recomendaremos canciones parecidas!")
  elif cancion_fav=="ver menu" or cancion_fav=="Ver menu":
    print("................................\n\n", (df["song"].head(10)),"\n.................................\n")
    pedir_cancion()
  else:
     print("Lo sentimos, esa canción no se encuentra en nuestra base de datos. Inténtalo de nuevo:")
     pedir_cancion()

pedir_cancion()



Ingresa tu canción favorita, o ingrese 'Ver menu' para ver una muestra de canciones disponibles 
 ¡Recuerda respetar las mayúsculas y los espacios de tu canción!
 Bye Bye Bye
Excelente, en un momento te recomendaremos canciones parecidas!


In [ ]:
# definimos las variables a comparar
caracteristicas_cancion = dflimpio[dflimpio["song"] == cancion_fav][["energy", "loudness", "tempo"]].values
genero_esperado = dflimpio[dflimpio["song"] == cancion_fav]["genre"].iloc[0]

# usamos una librería para comparar las canciones
dflimpio["similarity"] = dflimpio.apply(lambda row: cosine_similarity(caracteristicas_cancion.reshape(1, -1),
                                                           row[["energy", "loudness", "tempo"]].values.reshape(1, -1))[0][0], axis=1)
#nos aseguramos de que las canciones sugeridas sean del mismo género que la original
dflimpio = dflimpio[dflimpio["genre"].str.split(", ").apply(lambda x: genero_esperado in x)]

# Ordenamos nuestro dataframe
canciones_parecidas = dflimpio.sort_values(by="similarity", ascending=False)
canciones_parecidas.reset_index(inplace=True)
canciones_parecidas=canciones_parecidas.drop(["index","similarity"], axis=1)
canciones_parecidas = canciones_parecidas.iloc[1: , :]

# Imprimimos nuestro nuevo dataframe!
print("¡Listo! Tenemos una lista de canciones parecidas que te pueden interesar:\n\n")
canciones_parecidas.head(15)

¡Listo! Tenemos una lista de canciones parecidas que te pueden interesar:




,artist,song,duration_ms,explicit,year,popularity,energy,loudness,tempo,genre
1,Benny Benassi,Satisfaction (Isak Original Extended) - Benny ...,285570,False,2003,66,0.698,-3.626,130.017,"pop, Dance/Electronic"
2,Flo Rida,GDFR (feat. Sage the Gemini & Lookas),190185,False,2015,49,0.827,-4.036,145.889,"hip hop, pop"
3,Wiz Khalifa,Black and Yellow,217666,True,2011,75,0.834,-4.524,164.020,"hip hop, pop"
4,Avril Lavigne,When You're Gone,240493,False,2007,68,0.719,-3.921,142.034,pop
5,*NSYNC,It's Gonna Be Me,191040,False,2000,60,0.874,-4.666,165.090,pop
6,Clean Bandit,Tears (feat. Louisa Johnson),225914,False,2016,60,0.770,-3.645,130.037,"pop, Dance/Electronic"
7,Logic,Everyday,204746,True,2018,69,0.741,-4.099,149.908,"hip hop, pop"
8,Britney Spears,Toxic,198800,False,2003,81,0.838,-3.914,143.040,pop
9,Kelly Clarkson,My Life Would Suck Without You,211493,False,2009,68,0.882,-4.006,144.982,"pop, R&B"
10,Bazzi,Mine,131064,True,2018,75,0.789,-3.874,142.929,"pop, Dance/Electronic"
